

# VARIATIONS TEMPORELLES DU BRUIT EN ANTARCTIQUE

---

**Description du Notebook :**
> Dans ce notebook est étudié comment le bruit sismique à évolué avec les différentes mesures sanitaires mises en place par le gouvernement.


---
# Importation des modules

In [1]:
%matplotlib inline

import datetime
import os
from glob import glob

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import tqdm.notebook as tqdm
import warnings

from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNNoDataException
from obspy.signal import PPSD

%load_ext autoreload
%autoreload 2
from seismic_noise import *
import seismosocialdistancing

---
# 1. Récupération des données

### Paramètres initiaux

In [2]:
### Paramètres généraux
load_data = False              # Recalculer les PPSDs et le déplacement RMS
freqs = [(2.0,8.0)]          # Bandes de fréquences à étudier 
st_file = "DATA/st_metadata/stations_antarctic.txt"  # Fichier de stations
path = "C:/Users/minid/Documents/Travail"
noise_input_data = "B:/SeismicNoiseData/"
timeseries_data = "Files/noise_timeseries/"
PPSD_FOLDER  = path + "/PPSD"
DRMS_FOLDER  = path + "/DRMS"
MSEED_FOLDER = path + "/MSEED"
FIG_FOLDER = "RESULTS"
tStart = "2010-12-01"
tEnd = "2021-09-04"

### Conversion des périodes en UTCDateTime
periods = [[UTCDateTime(tStart), UTCDateTime(tEnd)]]

### Récupération des PPSDs et du déplacement RMS

In [3]:
st_name = np.loadtxt(st_file, dtype=str)
st_name = ["G.CCD.00.BHZ_2_8Hz"]   # Station à étudier
dRMS = {}
for st_str in st_name:
    try:
        dataframe = pd.read_csv("{}/{}.csv".format(DRMS_FOLDER, st_str), index_col=0)
        dRMS[st_str] = dataframe
    except:
        pass
st_name = list(dRMS.keys())

In [4]:
dRMS.keys()

dict_keys(['G.CCD.00.BHZ_2_8Hz'])

In [5]:
freq="2.0-8.0"
noise = dRMS[st_str][freq]
t, h, noise_matrix = hourly_var(noise, periods)



In [6]:
print(len(noise_matrix))

48


---
# 2. Variations horaires du bruit

## 2.1 Calcul de variations sur toute la période donnée

#### Tags à afficher sur le graphique

In [7]:
#décalage horaire été CCD
deca_hor=7

#décalage horaire été DRV
#deca_hor=10

#indice pour concatener la matrice pour prendre en compte le décalage horaire
indice=48-deca_hor*2


## 2.2 Calcul de variations horaires sur plusieurs sous-périodes

In [8]:
#print(dRMS)

In [ ]:
### Paramètres à changer
freq = "2.0-8.0"
st_name = "G.CCD.00.BHZ_2_8Hz"
periods = [[[UTCDateTime("2020-11-01"),UTCDateTime("2021-02-01")]],\
           [[UTCDateTime("2020-11-08"),UTCDateTime("2020-11-09")]],\
           [[UTCDateTime("2020-12-25"),UTCDateTime("2020-12-26")],[UTCDateTime("2019-12-25"),UTCDateTime("2019-12-26")],[UTCDateTime("2018-12-25"),UTCDateTime("2018-12-26")],[UTCDateTime("2017-12-25"),UTCDateTime("2017-12-26")],[UTCDateTime("2016-12-25"),UTCDateTime("2016-12-26")],[UTCDateTime("2015-12-25"),UTCDateTime("2015-12-26")],[UTCDateTime("2014-12-25"),UTCDateTime("2014-12-26")],[UTCDateTime("2013-12-25"),UTCDateTime("2013-12-26")],[UTCDateTime("2012-12-25"),UTCDateTime("2012-12-26")]],\
           [[UTCDateTime("2020-12-24"),UTCDateTime("2020-12-25")],[UTCDateTime("2019-12-24"),UTCDateTime("2019-12-25")],[UTCDateTime("2018-12-24"),UTCDateTime("2018-12-25")],[UTCDateTime("2017-12-24"),UTCDateTime("2017-12-25")],[UTCDateTime("2016-12-24"),UTCDateTime("2016-12-25")],[UTCDateTime("2015-12-24"),UTCDateTime("2015-12-25")],[UTCDateTime("2014-12-24"),UTCDateTime("2014-12-25")],[UTCDateTime("2013-12-24"),UTCDateTime("2013-12-25")],[UTCDateTime("2012-12-24"),UTCDateTime("2012-12-25")]],\
           [[UTCDateTime("2017-12-31"),UTCDateTime("2018-01-01")]],\
           [[UTCDateTime("2019-12-03"),UTCDateTime("2019-12-04")]]]


new_median=[]

for i in periods :
    if len(i)<=1:
        name = "{}{}_{}".format(st_name, i[0][0].strftime('%Y-%m-%d'), i[0][1].strftime('%Y-%m-%d'))
        noise = dRMS[st_name][freq]
        t, h, noise_matrix = hourly_var(noise, i)
        median1 = np.nanmedian(noise_matrix, axis=1)
        new_median.append(median1.tolist()[indice:] + median1.tolist()[0:indice])

    else :
        n=[]
        for v in i:
            noise = dRMS[st_name][freq]
            t, h, noise_matrix_inter = hourly_var(noise, [v])
            n.append(noise_matrix_inter)
    
        noise_matrix=np.concatenate((n[0],n[1]),axis=1)
        for k in range (1,len(n)-1):
            noise_matrix=np.concatenate((noise_matrix,n[k+1]),axis=1)                                   
        median1 = np.nanmedian(noise_matrix, axis=1)
        new_median.append(median1.tolist()[indice:] + median1.tolist()[0:indice])
        

plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams["axes.facecolor"] = 'w'
plt.rcParams.update({'font.size': 22})
plt.title('Hourly displacement comparison (2-8Hz)')
plt.xlabel('Hours')
plt.ylabel('Displacement (log(m))' )
liste=('Summer','Day off','Christmas 25','Christmas 24','New Year 31','Raid')
ind=0
for i in new_median :
    plt.yscale('log')
    plt.plot(h,i,label=liste[ind])
    ind=ind+1

plt.xticks(np.arange(0, 24, 1), np.arange(0, 24, 1))
plt.grid()
plt.legend(fontsize=22)
plt.show()
print(type(median1))

In [ ]:
tStart = '2021-01-01'
tEnd = '2021-01-30'

st_str = str('G.CCD.00.BHZ')

ppsds = PPSD.load_npz("{}/{}/{}_{}.npz".format(PPSD_FOLDER, st_str,tStart,st_str))
datelist = pd.date_range(start=UTCDateTime(tStart).datetime,end=UTCDateTime(tEnd).datetime)

for day in datelist:
    datestr = day.strftime("%Y-%m-%d")
    ppsds.add_npz("{}/{}/{}_{}.npz".format(PPSD_FOLDER, st_str,datestr,st_str))
    
    
ppsds.plot_spectrogram(clim=(-160,-100))